### Connect To Workspace

In [1]:
from azureml.core import Workspace

workspace = Workspace.from_config()
datastore = workspace.get_default_datastore()

### Get dataset and compute target

In [2]:
pokemon_dataset = datastore.path('pokemon-data')
compute_target = workspace.compute_targets['k80cluster']

### Create experiment

In [3]:
from azureml.core import Experiment

experiment_name = 'pokemon-classifier'
experiment = Experiment(workspace, name=experiment_name)

### Submit Job

In [6]:
%pycat training/train.py

In [21]:
from azureml.train.estimator import Estimator

estimator = Estimator(source_directory='training',
                      entry_script='train.py',
                      compute_target=compute_target,
                      script_params = {
                          '--data_dir': pokemon_dataset.as_mount(),
                          '--image_dim': 128,
                          '--learning_rate': 0.001,
                          '--batch_size': 32,
                          '--steps_per_epoch': 25,
                          '--num_epochs': 20,
                          '--output_dir':'./outputs',
                      },
                      use_gpu=True,
                      pip_packages=['tensorflow-gpu==2.0.0', 'Pillow==6.2.0', 'scipy==1.1.0'])

In [22]:
run = experiment.submit(estimator)

In [23]:
from azureml.widgets import RunDetails
RunDetails(run).show()

_UserRunWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', '…

### Register model

In [24]:
model = run.register_model(model_name='pokemon-classifier', model_path='outputs')